In [1]:
from typing import List, Dict, Set, Tuple

In [2]:
# Definition for TreeNode.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        
    def printTree(self) -> list:
        result = list()
        thislevel = [self]
        while thislevel:
            nextlevel = list()
            none_list=1
            for n in thislevel:
                if n !=None:
                    none_list=0
                    break
            if none_list==1:
                return result
                        
            for n in thislevel:
                if n != None: 
                    result.append(n.val)
                    nextlevel.append(n.left)
                    nextlevel.append(n.right)
                else:
                    result.append(None)
                    nextlevel.append(None)
                    nextlevel.append(None)
                    
            thislevel = nextlevel
        return result

In [3]:
#Create a BST from a list, then return root node
from collections import deque

def create_linked_bst(arr: list) -> TreeNode:
    if len(arr) < 1: 
        return None
    n = iter(arr)
    tree = TreeNode(next(n))
    fringe = deque([tree])
    
    while True:
        head = fringe.popleft()
        try:
            l = next(n)
            head.left = TreeNode(l) if l != None else None
            fringe.append(head.left)
            r = next(n)
            head.right = TreeNode(r) if r != None else None
            fringe.append(head.right)
        except StopIteration:
            break
    return tree

# P1. Range sum of BST 

Binary Search Tree에서 range `[low, high]`에 해당하는 BST의 모든 노드 값을 합산하여 반환하는 함수를 작성하여라. 

* BST의 각 Node는 `TreeNode` class로 정의되며, 이 정의는 `BST_Helper.py` 파일을 확인하면 된다. `printTree()` Method를 호출하면 tree의 각 노드의 값을 리스트로 만들어 Return해준다. 

* Input은 BST의 `Root` Node와 `low, high`가 주어질 것이다. 

* 모든 node의 값은 BST 안에서 유일하다. 

* 코드 구현 확인을 위하여 `BST_Helper.py` 파일 내부에 아래 함수가 구현되어 있다. 
    - `create_linked_bst`: 정수 `List`를 입력받아 BST를 구성하고 `root` node를 반환. 이때 정수 `List`는 첫 element를 `root` node의 값, 두번째는 `root` node의 왼쪽 값, 세번째는 `root` node의 오른쪽 값… 으로 Breadth first traversal의 순서로 입력되어야 한다.  또한 가지에 해당하는 값이 없을 경우 `None`을 입력한다. 

     - 예를 들어, 다음 `list`를 입력하면 `[10,5,15,3,7,None,18]` 아래와 같은 BST를 생성하고 `root` node를 return 한다.
     ```
         10 
         / \ 
        5  15 
       / \   \ 
      3   7  18 
     ```
     
* BST의 특성을 이용하지 않고 `List`의 값 중에서 범위를 체크하여 합산하는 방식으로 해결하는 것은 금지한다. 이를 위해 BST의 Child의 조건이 위배된 형태로 Input이 주어질 수 있다.  
    - left node `y` of node `x` has value less than `x`’s value 
    - right node `y` of node `x` has value greater than `x`’s value
    - 즉, child node의 경우는 해당 노드의 값보다 크거나 작아야 하지만 grand  children부터는 이 조건에 구애를 받지 않는다. 예를 들어 아래와 같은 형태의 BST도 Input으로 주어질 수 있다. 9는 10보다 작지만 오른쪽 Subtree에 위치한다.  이와 같은 Input이 주어질 때 계산방법은 아래 예시 1, 예시 2를 참고하라. 

```
    10 
   /   \ 
  5    15 
 / \   / \ 
3   7 9  18
```
예시 1) 3, 5, 7 이 해당 range안에 있으므로 15가 return 된다. 9는 Range 안에 있으나 BST를 검색할 때 range의 최대값 9가 Root node보다 작으므로 왼쪽 Subtree만 검색하여 3,5,7의 합만 Return 하여야 한다.
```py
>>> root = create_linked_bst([10,5,15,3,7,9,18]) 
>>> P1(root, 3, 9) 
15
```

예시 2) 3, 5, 7, 9, 10, 15가 해당 range 안에 있으므로 49가 return된다. range의 최대값 15 가 Root node보다 크므로 왼쪽, 오른쪽 Subtree를 모두 검색하여야 하고 9는 합산시 포함되어야 한다.
```py
>>> root = create_linked_bst([10,5,15,3,7,9,18]) 
>>> P1(root, 3, 15) 
49
```

예시 3) 7, 10, 15가 해당 range 안에 있으므로 32가 return된다. 
```py
>>> root = create_linked_bst([10,5,15,3,7,None,18]) 
>>> P1(root, 7, 15) 
32
```

예시 4) 6, 7, 10이 해당 range 안에 있으므로 23이 return된다. 
```
     10 
     / \ 
    5  15 
   / \   \ 
  3   7  18 
 /   /  
1   6
```
```py
>>> root = create_linked_bst ([10,5,15,3,7,13,18,1,None,6]) 
>>> P1(root, 6, 10) 
23 
```

* Recursion 쓸 때 주의할 점 -> base case와 original function은 그 return type이 동일해야 한다!
    - base case에서 nothing return $\Leftrightarrow$ original function도 nothing return
    - base case에서 return $\Leftrightarrow$ original function도 같은 type return

In [4]:
def P1(root:TreeNode, low:int, high:int) -> int:
    ans = 0
    if root is None:
        return 0
    if low <= root.val <= high:
        ans = root.val
        print(ans)
    if root.val >= low:
        ans += P1(root.left, low, high)
    if root.val <= high:
        ans += P1(root.right, low, high)
    return ans

#     def dft(root:TreeNode):
#         ans = 0
#         if root is None:
#             return 0
#         if low <= root.val <= high:
#             ans = root.val
#         if root.val <= high:
#             ans += dft(root.right)
#         if root.val >= low:
#             ans += dft(root.left)
# #         print(ans)
#         return ans

#     return dft(root)

In [5]:
# 모든 값을 values라는 list에 받는 방법
def P1(root:TreeNode, low:int, high:int) -> int:
    values = [] # recursion과 무관한 값이므로 새로운 함수가 필요
    
#     def dft(root:TreeNode, result:list):
#         if root is None:
#             return result
#         if low <= root.val <= high:
#             result.append(root.val)
#         if root.val >= low:
#             dft(root.left, result)
#         if root.val <= high:
#             dft(root.right, result)
#         return result

    def dft(root:TreeNode):
        if root is None:
            return
        if low <= root.val <= high:
            values.append(root.val)
        if root.val >= low:
            dft(root.left)
        if root.val <= high:
            dft(root.right)
    
    dft(root)
    
    return sum(values)

In [6]:
root = create_linked_bst([10,5,15,3,7,9,18])

In [7]:
P1(root, 3, 9)

15

In [8]:
P1(root, 3, 15)

49

In [9]:
root = create_linked_bst([10,5,15,3,7,None,18])
P1(root, 7, 15)

32

In [10]:
root = create_linked_bst ([10,5,15,3,7,13,18,1,None,6])
P1(root, 6, 10)

23

# P2 Binary Tree Level Order Traversal 

Binary tree에 대하여 Bottom-up level order traversal을 하고 return하는 함수를 작성하시오. 즉, Leaf부터 Root까지 각 level에 대해, 왼쪽에서부터 오른쪽 순서로 저장되어야 한다. 

* 이중 리스트 형태로 return해야 하며, 같은 Depth인 Node들의 값이 같은 List에 저장되어야 한다. 

* input으로는 binary tree의 `root`가 주어진다. 

예시 1)
```
    3 
   / \ 
  9  20 
 /     \ 
15      7 
```
```py
>>> root = create_linked_bst ([3,9,20,None,None,15,7]) 
>>> P2(root) 
[[15, 7], [9, 20], [3]]
```

예시 2) 
```
    10 
   /  \ 
  5   15 
 / \    \ 
3   7   18
```
```py
>>> root = create_linked_bst([10,5,15,3,7,None,18]) 
>>> P2(root) 
[[3, 7, 18], [5, 15], [10]] 
```

예시 3)
```py
>>> root = create_linked_bst([5,3,6,2,4,None,7]) 
>>> P2(root) 
[[2, 4, 7], [3, 6], [5]] 
```

In [11]:
def P2(root:TreeNode) -> List[List[int]]:
    def levelOrder(root:TreeNode) -> List[List[int]]:
        visited = []
        if not root:
            return visited

        level_dict = dict()
        curNode = root
        queue = [root]
        level_dict[root] = 0
        levels = [0]
        while queue:
            curNode = queue.pop(0)
            if curNode.left:
                level_dict[curNode.left] = level_dict[curNode] + 1
                queue.append(curNode.left)
                if level_dict[curNode.left] not in levels:
                    levels.append(level_dict[curNode.left])
            if curNode.right:
                level_dict[curNode.right] = level_dict[curNode] + 1
                queue.append(curNode.right)
                if level_dict[curNode.right] not in levels:
                    levels.append(level_dict[curNode.right])
        
        for level in sorted(levels, reverse=True):
            level_list = []
            for key in level_dict.keys():
                if level_dict[key] == level:
                    level_list.append(key.val)
            visited.append(level_list)
        
        return visited

    return levelOrder(root)

In [12]:
root = create_linked_bst ([3,9,20,None,None,15,7])
P2(root)

[[15, 7], [9, 20], [3]]

In [13]:
root = create_linked_bst([10,5,15,3,7,None,18]) 
P2(root)

[[3, 7, 18], [5, 15], [10]]

In [14]:
root = create_linked_bst([5,3,6,2,4,None,7]) 
P2(root) 

[[2, 4, 7], [3, 6], [5]]

# P3. Insert value into BST : make full BST 

BST에서 모든 Depth의 모든 Node에 값이 있는 경우 Full BST라고 하자. 예를 들면 아래와 같은 BST를 지칭한다. 

<img src="./hw10_instruction-pdf-1.png">

Full BST에서 1개의 Node만 비어 있는 BST에 1개의 값을 추가하여 Full BST로 만들고, root node를 return하는 함수를 작성하여라. 
* Input은 BST의 `root` node와 추가할 정수값이다. 
* Node의 모든 값은 정수라고 가정한다. 
* 추가되는 값은 Input BST에 존재하지 않는다고 가정한다.
* Tree를 새로 생성하는 방식으로의 구현은 인정하지 않는다. (ex. 원래 tree의 값들을 받아 리스트로 만든 뒤 다음 값을 추가해서 create_linked_bst 를 사용하는 방식의 구현은 안됨) 

예시 1) 왼쪽의 BST에 6을 넣을 경우 오른쪽과 같은 BST를 Return해야 한다. 

<img src="./hw10_instruction-pdf-2.png">

```py
>>> root = create_linked_bst([7,3,8,2,5,None,9]) 
>>> fullBST = P3(root, 6) 
>>> print(fullBST.printTree()) 
[6, 3, 8, 2, 5, 7, 9]
```
예시 2) 왼쪽의 BST에 10을 넣을 경우 오른쪽과 같은 BST를 Return해야 한다. 

<img src="./hw10_instruction-pdf-6.png">

```py
>>> root = create_linked_bst( [7,3,8,2,5,None,9]) 
>>> fullBST = P3(root, 10) 
>>> print(fullBST.printTree()) 
[7, 3, 9, 2, 5, 8, 10] 
```
예시 3) 왼쪽의 BST에 7을 넣을 경우 오른쪽과 같은 BST를 Return해야 한다. 
<img src="./hw10_instruction-pdf-3.png">
```py
>>> root = create_linked_bst([10,5,15,3,6,12,18,1,4,None,8,11,13,16,20])
>>> fullBST = P3(root, 7) 
>>> print(fullBST.printTree()) 
[10, 5, 15, 3, 7, 12, 18, 1, 4, 6, 8, 11, 13, 16, 20] 
```

예시 4) 왼쪽의 BST에 14를 넣을 경우 오른쪽과 같은 BST를 Return해야 한다. 

<img src="./hw10_instruction-pdf-4.png">

```py
>>> root = create_linked_bst([10,5,15,3,7,12,18,1,4,6,8,11,13,None,20]) 
>>> fullBST = P3(root, 14) 
>>> print(fullBST.printTree()) 
[10, 5, 14, 3, 7, 12, 18, 1, 4, 6, 8, 11, 13, 15, 20] 
```
예시 5) 왼쪽의 BST에 9를 넣을 경우 오른쪽과 같은 BST를 Return해야 한다. 

<img src="./hw10_instruction-pdf-5.png">

```py
>>> root = create_linked_bst([10,5,15,3,7,12,18,1,4,6,8,11,13,None,20]) 
>>> fullBST = P3(root, 9) 
>>> print(fullBST.printTree()) 
[9, 5, 13, 3, 7, 11, 18, 1, 4, 6, 8, 10, 12, 15, 20]
```

In [15]:
def P3(root: TreeNode, val: int) -> TreeNode:    
    ##### Write your Code Here #####

    def __balancedBST(nodes: list, start: int, end: int) -> TreeNode:
        """
        1. 중앙 노드 선택: 주어진 정렬된 노드 리스트에서 중앙에 위치한 노드를 선택합니다. 이 노드가 현재 서브트리의 루트 노드가 됩니다. 
           중앙 노드는 (start + end) // 2로 계산됩니다.
        2. 왼쪽 서브트리 생성: 중앙 노드의 왼쪽에 있는 노드들을 이용하여 왼쪽 서브트리를 생성합니다. 
           이를 위해 __balancedBST 함수를 재귀적으로 호출하며, start부터 mid-1까지의 노드들을 사용합니다.
        3. 오른쪽 서브트리 생성: 중앙 노드의 오른쪽에 있는 노드들을 이용하여 오른쪽 서브트리를 생성합니다. 
           이를 위해 __balancedBST 함수를 재귀적으로 호출하며, mid+1부터 end까지의 노드들을 사용합니다.
        4. 서브트리 연결: 중앙 노드의 left 포인터를 왼쪽 서브트리의 루트로, right 포인터를 오른쪽 서브트리의 루트로 설정합니다.
        5. 루트 반환: 현재 서브트리의 루트 노드를 반환합니다.
        이 원리를 기반으로, 함수는 주어진 정렬된 노드 리스트를 이용하여 균형잡힌 이진 탐색 트리를 생성합니다. 중앙 노드를 루트로 선택하는 것은 BST의 특성을 유지하기 위함입니다. 중앙 노드를 기준으로 왼쪽에는 더 작은 값들, 오른쪽에는 더 큰 값들이 위치하게 되므로, 이를 이용하여 재귀적으로 균형잡힌 BST를 구성하게 됩니다.
        """
        if start > end:
            return None

        mid = (start + end) // 2
        node = nodes[mid]

        node.left = __balancedBST(nodes, start, mid-1)
        node.right = __balancedBST(nodes, mid+1, end)

        return node
    
    def __insertBST(root: TreeNode, val: int):
        """
        1. 기본 케이스: 만약 현재 루트(root)가 None이면, 새로운 노드를 생성하고 그 값을 반환합니다.
        2. 재귀 케이스: 현재 루트의 값이 삽입하려는 값보다 작다면, 오른쪽 서브트리에 값을 삽입해야 합니다. 
           그렇지 않다면 왼쪽 서브트리에 삽입합니다. 이를 위해 해당 서브트리의 루트를 대상으로 __insertBST 함수를 재귀적으로 호출합니다.
        3. 마지막으로, 현재 루트를 반환합니다. 이는 현재 노드 아래의 서브트리에 변화가 있을 수 있기 때문입니다.
        """
        if root == None:
            return TreeNode(val)

        else:
            if root.val < val:
                root.right = __insertBST(root.right, val)
            else:
                root.left = __insertBST(root.left, val)

        return root

    def __storeBST(root: TreeNode, nodes: list):
        """
        이 함수는 주어진 BST의 모든 노드를 중위 순회 방식으로 방문하며, 노드들을 주어진 리스트(nodes)에 추가하는 재귀 함수입니다.
        원리:
        1. 중위 순회: BST의 중위 순회는 노드들을 오름차순으로 방문합니다. 
           따라서, __storeBST 함수는 먼저 왼쪽 서브트리를 방문, 그 다음 현재 노드를 리스트에 추가, 
           그리고 오른쪽 서브트리를 방문하는 순서로 작동합니다.
        2. 이 방식으로, nodes 리스트는 BST의 모든 노드를 오름차순으로 포함하게 됩니다.
        """
        if root == None:
            return
        __storeBST(root.left, nodes)
        nodes.append(root)
        __storeBST(root.right, nodes)

    root = __insertBST(root, val)
    nodes = []
    __storeBST(root, nodes)
    
    n = len(nodes)
    print([node.val for node in nodes])
    return __balancedBST(nodes, 0, n-1)
    ##### End of your code #####

In [16]:
>>> root = create_linked_bst([7,3,8,2,5,None,9]) 
>>> fullBST = P3(root, 6) 
>>> print(fullBST.printTree()) 

[2, 3, 5, 6, 7, 8, 9]
[6, 3, 8, 2, 5, 7, 9]


In [17]:
>>> root = create_linked_bst([10,5,15,3,7,12,18,1,4,6,8,11,13,None,20]) 
>>> fullBST = P3(root, 9) 
>>> print(fullBST.printTree()) 

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 18, 20]
[9, 5, 13, 3, 7, 11, 18, 1, 4, 6, 8, 10, 12, 15, 20]


In [18]:
## BFS
def BFS(root:TreeNode) -> List:
    visited = []
    queue = [root]
    visited = []
    while queue:
        curr = queue.pop(0)
        if curr not in visited:
            visited.append(curr)
        if curr.left:
            queue.append(curr.left)
        if curr.right:
            queue.append(curr.right)
    return [node.val for node in visited]

BFS(fullBST)

[9, 5, 13, 3, 7, 11, 18, 1, 4, 6, 8, 10, 12, 15, 20]

In [28]:
def PreOrder(root:TreeNode) -> List:
    visited = []
    stack = [root]
    curr = root
    while stack:
        curr = stack.pop(-1)
        if curr not in visited:
            visited.append(curr)
        if curr.right:
            stack.append(curr.right)
        if curr.left:
            stack.append(curr.left)
    return [node.val for node in visited]

print(PreOrder(fullBST))

[9, 5, 3, 1, 4, 7, 6, 8, 13, 11, 10, 12, 18, 15, 20]


In [29]:
def PreOrder(root:TreeNode) -> List:
    visited = []
    def dfs(root:TreeNode):
        visited.append(root)
        if root.left:
            dfs(root.left)
        if root.right:
            dfs(root.right)
    dfs(root)
    return [node.val for node in visited]
print(PreOrder(fullBST))

def PreOrder(root:TreeNode) -> List:
    visited = []
    def dfs(root:TreeNode):
        if root is None:
            return
        visited.append(root)
        dfs(root.left)
        dfs(root.right)
    dfs(root)
    return [node.val for node in visited]
print(PreOrder(fullBST))

[9, 5, 3, 1, 4, 7, 6, 8, 13, 11, 10, 12, 18, 15, 20]
[9, 5, 3, 1, 4, 7, 6, 8, 13, 11, 10, 12, 18, 15, 20]


In [35]:
def InOrder(root:TreeNode) -> List:
    visited = []
    def dfs(root:TreeNode):
        if root.left:
            dfs(root.left)
        visited.append(root)
        if root.right:
            dfs(root.right)
    dfs(root)
    return [node.val for node in visited]
print(InOrder(fullBST))

def InOrder(root:TreeNode) -> List:
    visited = []
    def dfs(root:TreeNode):
        if root is None:
            return
        dfs(root.left)
        visited.append(root)
        dfs(root.right)
    dfs(root)
    return [node.val for node in visited]
print(InOrder(fullBST))

def InOrder(root:TreeNode) -> List:
    visited = []
    stack = []
    curr = root
    # 스택이 비어있지 않거나 현재 노드가 None이 아닌 동안 반복
    while stack or curr:
        # 현재 노드가 None이 아니면 스택에 추가하고 왼쪽 자식으로 이동
        while curr:
            stack.append(curr)
            curr = curr.left
        
        # 스택의 마지막 요소를 꺼내서 방문 리스트에 추가
        curr = stack.pop(-1)
        visited.append(curr)
        
        # 오른쪽 자식으로 이동
        curr = curr.right
    return [node.val for node in visited]
print(InOrder(fullBST))

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 18, 20]
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 18, 20]
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 18, 20]


In [36]:
def PostOrder(root:TreeNode) -> List:
    visited = []
    def dfs(root:TreeNode):
        if root.left:
            dfs(root.left)
        if root.right:
            dfs(root.right)
        visited.append(root)
    dfs(root)
    return [node.val for node in visited]
print(PostOrder(fullBST))

def PostOrder(root:TreeNode) -> List:
    visited = []
    def dfs(root:TreeNode):
        if root is None:
            return
        dfs(root.left)
        dfs(root.right)
        visited.append(root)
    dfs(root)
    return [node.val for node in visited]
print(PostOrder(fullBST))

def PostOrder(root:TreeNode) -> List:
    visited = []
    stack = [(root, False)]
    while stack:
        curr, visited_left = stack.pop()
        if curr:
            if visited_left:
                visited.append(curr)
            else:
                stack.append((curr, True))
                stack.append((curr.right, False))
                stack.append((curr.left, False))
    return [node.val for node in visited]
print(PostOrder(fullBST))

[1, 4, 3, 6, 8, 7, 5, 10, 12, 11, 15, 20, 18, 13, 9]
[1, 4, 3, 6, 8, 7, 5, 10, 12, 11, 15, 20, 18, 13, 9]
[1, 4, 3, 6, 8, 7, 5, 10, 12, 11, 15, 20, 18, 13, 9]
